In [ ]:
!pip install datasets

from datasets import load_dataset
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
from collections import Counter
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
import seaborn as sns
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from collections import Counter
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud
from collections import Counter
import numpy as np
import tensorflow as tf
from tensorflow import keras
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import classification_report
import seaborn as sns
import matplotlib.pyplot as plt
from pprint import pprint
# Download NLTK stop words
nltk.download('stopwords')
nltk.download('punkt')

!pip install -q transformers==4.37.2
!pip install pydot

from transformers import AutoTokenizer, TFAutoModel, TFAutoModelForMaskedLM

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
dataset = load_dataset("orieg/elsevier-oa-cc-by")
elsevier_dataset = dataset

/usr/local/lib/python3.10/dist-packages/datasets/load.py:1461: FutureWarning: The repository for orieg/elsevier-oa-cc-by contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/orieg/elsevier-oa-cc-by
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


In [ ]:
train_set = elsevier_dataset['train']
train_df = pd.DataFrame(train_set)

val_set = elsevier_dataset['validation']
val_df = pd.DataFrame(val_set)

test_set = elsevier_dataset['test']
test_df = pd.DataFrame(test_set)

In [ ]:
#subject areas based on SciBERT training data
relevant_subject_areas = {'COMP', 'BIOC', 'MEDI'}

def filter_relevant_subjects(subjareas_list):
    return [area for area in subjareas_list if area in relevant_subject_areas]

train_df['subjareas'] = train_df['subjareas'].apply(filter_relevant_subjects)
val_df['subjareas'] = val_df['subjareas'].apply(filter_relevant_subjects)
test_df['subjareas'] = test_df['subjareas'].apply(filter_relevant_subjects)

train_df = train_df[train_df['subjareas'].map(bool)]
val_df = val_df[val_df['subjareas'].map(bool)]
test_df = test_df[test_df['subjareas'].map(bool)]

In [ ]:
def prepare_elsevier_data(train_df, val_df, test_df):
    """
    Prepares the Elsevier corpus splits for multi-label classification.
    """
    mlb = MultiLabelBinarizer()

    all_labels = pd.concat([train_df['subjareas'], val_df['subjareas'], test_df['subjareas']])
    mlb.fit(all_labels)

    train_labels = mlb.transform(train_df['subjareas'])
    val_labels = mlb.transform(val_df['subjareas'])
    test_labels = mlb.transform(test_df['subjareas'])

    train_texts = train_df['abstract'].tolist()
    val_texts = val_df['abstract'].tolist()
    test_texts = test_df['abstract'].tolist()

    return train_texts, val_texts, test_texts, train_labels, val_labels, test_labels, mlb.classes_

train_texts, val_texts, test_texts, train_labels, val_labels, test_labels, target_names = prepare_elsevier_data(train_df, val_df, test_df)

In [ ]:
model_checkpoint = 'allenai/scibert_scivocab_cased'
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [ ]:
train_encodings = tokenizer(train_texts, truncation=True, padding='max_length', max_length=300)
val_encodings = tokenizer(val_texts, truncation=True, padding='max_length', max_length=300)
test_encodings = tokenizer(test_texts, truncation=True, padding='max_length', max_length=300)

In [ ]:
def build_finetune_scibert(pretrained_model_name_or_path='allenai/scibert_scivocab_cased', num_classes=3, max_length=300):
    """
    Builds a fine-tuned SciBERT model.
    """
    tf.keras.backend.clear_session()

    # Load the pre-trained SciBERT model
    scibert_model = TFAutoModel.from_pretrained(pretrained_model_name_or_path, from_pt=True)  # Note the from_pt=True

    # input layers
    input_ids = tf.keras.layers.Input(shape=(max_length,), dtype=tf.int32, name='input_ids')
    attention_mask = tf.keras.layers.Input(shape=(max_length,), dtype=tf.int32, name='attention_mask')

    # embeddings from the pre-trained model
    embeddings = scibert_model(input_ids=input_ids, attention_mask=attention_mask)[0]

    x = tf.keras.layers.GlobalAveragePooling1D()(embeddings)
    x = tf.keras.layers.Dense(50, activation='relu')(x)
    x = tf.keras.layers.Dropout(0.2)(x)

    # Output layer with sigmoid activation for multi-label classification
    output = tf.keras.layers.Dense(num_classes, activation='sigmoid')(x)

    model = tf.keras.Model(inputs=[input_ids, attention_mask], outputs=output)

    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=2e-5),
                  loss=tf.keras.losses.BinaryCrossentropy(),
                  metrics=['accuracy'])

    return model

scibert_model = build_finetune_scibert(num_classes=len(target_names))

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint

In [ ]:
def compute_sample_weights(df, label_column):
    """
    Compute sample weights for multi-label classification.
    """
    label_freq = pd.Series([label for sublist in df[label_column] for label in sublist]).value_counts()

    label_weights = 1 / label_freq
    label_weights /= label_weights.sum()
    label_weight_map = label_weights.to_dict()

    sample_weights = []

    for labels in df[label_column]:
        sample_weight = sum([label_weight_map[label] for label in labels if label in label_weight_map])
        normalized_weight = sample_weight / len(labels) if labels else 0
        sample_weights.append(normalized_weight)

    return np.array(sample_weights)

sample_weights = compute_sample_weights(train_df, 'subjareas')

In [ ]:
# convert encodings to numpy arrays....
train_inputs = {'input_ids': np.array(train_encodings['input_ids']), 'attention_mask': np.array(train_encodings['attention_mask'])}
val_inputs = {'input_ids': np.array(val_encodings['input_ids']), 'attention_mask': np.array(val_encodings['attention_mask'])}
test_inputs = {'input_ids': np.array(test_encodings['input_ids']), 'attention_mask': np.array(test_encodings['attention_mask'])}

history = scibert_model.fit(
    train_inputs,
    train_labels,
    validation_data=(val_inputs, val_labels),
    batch_size=8,
    epochs=2,
    sample_weight=sample_weights
)

Epoch 1/2


1520/1520 [==============================] - 280s 157ms/step - loss: 0.0734 - accuracy: 0.7647 - val_loss: 0.2965 - val_accuracy: 0.8043
Epoch 2/2
1520/1520 [==============================] - 222s 146ms/step - loss: 0.0515 - accuracy: 0.8223 - val_loss: 0.2752 - val_accuracy: 0.7768


In [ ]:
def show_results(features, true_labels, classes, threshold=0.5):
    """
    Display classification report and confusion matrices.
    """
    predictions = scibert_model.predict({'input_ids': features[0], 'attention_mask': features[1]})
    y_pred = (predictions >= threshold).astype(int)

    print(classification_report(true_labels, y_pred, target_names=classes, zero_division=0))

show_results([np.array(val_encodings['input_ids']), np.array(val_encodings['attention_mask'])], val_labels, target_names)


47/47 [==============================] - 10s 155ms/step
              precision    recall  f1-score   support

        BIOC       0.94      0.82      0.87       841
        COMP       0.78      0.75      0.77       220
        MEDI       0.83      0.90      0.86       704

   micro avg       0.87      0.84      0.86      1765
   macro avg       0.85      0.82      0.83      1765
weighted avg       0.87      0.84      0.86      1765
 samples avg       0.87      0.87      0.85      1765

